In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
response = requests.get('https://turbo.az')

In [4]:
soup = bs(response.content, 'html.parser')

## Avtomobil markalarının siyahısı hazırlanır

In [5]:
car = soup.find_all(id='q_make')
car_list = car[0].text.split('\n')[1:] # Avtomobil markalarının listi
#currency = soup.find_all(id='q_currency')
#currency_list = currency[0].text.split('\n') # Valyuta növlərinin listi"""
#region = soup.find_all(id='q_region')
#region_list = region[0].text.split('\n')[1:] # Şəhər və rayonların listi
#model = soup.find_all(id='q_model')
#model_list = model[0].text.split('\n')[1:] # Avtomobil modellərinin listi

## Saytdakı və hər gün əlavə olunan elanların sayı

In [6]:
info = soup.find_all(class_='ad-count--animation')
total = int(info[0].text.split()[0]) # Saytda olan bütün maşınların sayı
new = int(info[1].text.split()[1]) # Sayta hərgün əlavə olunan elanların sayı

## Avtomobillərin adları öz sıra nömrələri ilə uyğunlaşdırılır

In [7]:
value_list = [] # Hər bir avtomobil markasına uyğun gələn sıra nömrələrinin listi
car_dict = {} # Hər bir avtomobil markasına müvafiq sıra nömrəsi
for option in soup.find_all('option')[1:]:
    if option['value'] != 'azn':
        car_dict[option.text] = option['value']
        value_list.append(option['value'])
    else:
        break

## Hər bir avtmobilin URL-i

In [8]:
url_list = []
for c in car_dict:
    url = 'https://turbo.az/autos?utf8=✓&q%5Bmake%5D%5B%5D='+ car_dict[c] +'&q%5Bcurrency%5D=azn&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bloan%5D=0&q%5Bregion%5D%5B%5D=&commit=&q%5Bmodel%5D%5B%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=&q%5Bbarter%5D=0&q%5Bsort%5D='
    car_dict[c] = url
    url_list.append(url)

# DataFrame yaradılır və ilkin məlumatlar əlavə olunur

In [9]:
turbo = pd.DataFrame()
turbo['Firma'] = car_list
turbo['ID'] = value_list
#turbo['URL'] = url_list

## Model sayı

In [10]:
model_num = [] # Hər bi avtomobil markasının modellərinin sayı
for v in value_list:
    model = soup.find_all(class_=v)
    m_list = []
    for m in model[0:]:
        m_list.append(m.text)
        num = len(m_list)
    model_num.append(num)
turbo['Model sayı'] = model_num

## Markalar üzrə elan sayı

In [11]:
elan_list = []
for u in url_list:
    response = requests.get(u)
    soup = bs(response.content, 'html.parser')
    elan = soup.find_all(class_="products-title-amount")
    elan_num = int(elan[0].text.split()[0])
    elan_list.append(elan_num)
turbo['Elan sayı'] = elan_list

In [12]:
"""url1 = 'https://turbo.az/autos?utf8=✓&q%5Bmake%5D%5B%5D='
url2 = '&q%5Bcurrency%5D=azn&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bloan%5D=0&q%5Bregion%5D%5B%5D=&commit=&q%5Bmodel%5D%5B%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=&q%5Bbarter%5D=0&q%5Bsort%5D='
turbo['URL'] = url1 + turbo['ID'] + url2"""

"url1 = 'https://turbo.az/autos?utf8=✓&q%5Bmake%5D%5B%5D='\nurl2 = '&q%5Bcurrency%5D=azn&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bloan%5D=0&q%5Bregion%5D%5B%5D=&commit=&q%5Bmodel%5D%5B%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=&q%5Bbarter%5D=0&q%5Bsort%5D='\nturbo['URL'] = url1 + turbo['ID'] + url2"

In [13]:
turbo.sort_values('Elan sayı', ascending=False).head(10)

,Firma,ID,Model sayı,Elan sayı
84,Mercedes,4,260,6252
70,LADA (VAZ),5,33,4889
50,Hyundai,1,37,2267
95,Opel,29,19,1726
118,Toyota,23,32,1571
10,BMW,3,59,1363
65,Kia,8,31,1341
20,Chevrolet,41,23,961
35,Ford,2,24,936
94,Nissan,7,30,803
